In [ ]:
import os
os.getcwd()

### Gensim
1. It is a python library which provides a built-in models like  ready-to-use implementation of Word2Vec —you don’t have to build the neural network from scratch.
2. We can try the already built-in models like Word2Vec, GloVe, or FastText using gensim.downloader.api.load()
3. If we want to train our own embeddings we can do that - from gensim.models import Word2Vec
4. There are different word2vec models. If we want to create our own embedding model we need to import the Word2Vec class.

In [ ]:
pip install gensim

#### Let's see how we can load the google-news word2vec model

In [ ]:
import gensim
from gensim.models import Word2Vec, KeyedVectors

In [ ]:
import gensim.downloader as api
model = api.load("word2vec-google-news-300")

In [ ]:
# Let's see how the model give an output for a word
# Google news word2vec model has 300 dimensions and it is trained on 3 million words/phrases. 300 dimensions means each word is represented as a vector of 300 elements.
# Let's see the vector for the word 'king'
vector = model['king']
print(vector)

In [ ]:
vector_length = len(vector)
print(f"Length of the vector for the word 'king': {vector_length}")

#### Let's create our own Word2Vec model.

In [ ]:
# Read the data from the data folder
import pandas as pd
file_path = os.path.join('..', 'data', 'SMSSpamCollection.txt')
messages=pd.read_csv(file_path, sep='\t', names=['label', 'message'])

In [ ]:
messages

In [ ]:
messages.shape

In [ ]:
messages.head()

In [ ]:
# Lets do the data cleaning and preprocessing
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
ps = PorterStemmer()

In [ ]:
# This prepares text for CountVectorizer or TfidfVectorizer, which expect string sentences, not token lists.
# But for Word2Vec, we need list of token lists. That is done in the next cells.
corpus=[]
for i in range(0,len(messages)):
    # [^a-zA-Z] → “anything not (^) an uppercase (A–Z) or lowercase (a–z) letter”.THis this removes all digits (0–9), punctuation, symbols, etc. Replace them with spaces from message column
    # If we need numbers we can use [^a-zA-Z0-9]
    review = re.sub('[^a-zA-Z]', ' ', messages['message'][i])
    review = review.lower()
    # Split the sentences in to words by spaces
    review = review.split()
    # Apply the stemming
    review = [lemmatizer.lemmatize(word) for word in review]
    # Join the words to make sentences
    # If review is empty → ' '.join([]) → '' (an empty string).
    review = ' '.join(review)
    # Append the snetence to corpus
    corpus.append(review)

#### Meaning of the below list comprehensino
| Expression          | Meaning                                                         |
| ------------------- | --------------------------------------------------------------- |
| `map(len, corpus)`  | Gets length of each cleaned message                             |
| `zip(...)`          | Pairs lengths, cleaned messages, and original messages together |
| `if i < 1`          | Filters messages with empty cleaned text                        |
| `[ [i, j, k] ... ]` | Builds list showing length, cleaned text, and original text     |


In [ ]:
# Let's see if there are any empty messages in the corpus. 
# This helps in identifying the messages became empty after cleaning (e.g., removing stopwords, punctuation, etc.)
[[i,j,k] for i,j,k in zip(list(map(len,corpus)),corpus, messages['message']) if i<1]

#### empty strings '' can appear in corpus if the original message has no letters.

In [ ]:
# To get the complete details which original messages became empty after cleaning
for i, j, k in zip(list(map(len, corpus)), corpus, messages['message']):
    if i < 1:
        print(f"Original: {k}")
        print(f"Cleaned: '{j}'")
        print(f"Length: {i}\n")
        


In [ ]:
corpus

#### The 3 empty strings with white spaces that are after cleaning of the corpus will not be considered in the words. So total size will be reduced from 5572 to 5569 sentences.

##### How the below logic works?
| Case    | Will it be added to `words`?                             | Reason                                        |
| ------- | -------------------------------------------------------- | --------------------------------------------- |
| `''`    | ❌ No                                                     | `sent_tokenize` → `[]`                        |
| `'   '` | ❌ No (inner loop skips) or empty list if passed directly | No valid sentence/tokens                      |
| `'u'`   | ✅ Added as `[]` by default (filtered)                    | Single char → filtered out unless `min_len=1` |
| `'hi'`  | ✅ Added as `['hi']`                                      | Valid token                                   |



In [86]:
# Convert each cleaned sentence into a list of tokenized words for Word2Vec training.
from nltk.tokenize import sent_tokenize
from gensim.utils import simple_preprocess
# Words is the list of lists. It stores each sentence as a list of words and do it for all the sentences, for every sentence one list of words are created.
words=[]
for sent in corpus:
    sent_token=sent_tokenize(sent)
    for sent in sent_token:
        words.append(simple_preprocess(sent))

In [ ]:
words

##### Below are the steps to check the empty strings in words list and why there is a difference in the length of corpus and length of words

In [ ]:
any(len(w) == 0 for w in words)

In [ ]:
# Count empties
sum(len(w)==0 for w in words)

In [ ]:
# Check the indexes of the empty words
[i for i,w in enumerate(words) if len(w)==0]

In [ ]:
# This gives the 3 empty strings with white spaces from the corpus which I mentioned previously.
[i for i, x in enumerate(corpus) if x.strip() == '']

In [83]:
print(len(corpus))
print(len(words))

5572
5569


In [ ]:
for i in [4291, 5170]:
    print(f"Index {i}: {repr(corpus[i])}")

In [ ]:
words[4291]

In [ ]:
words[5170]

In [ ]:
# When you skip empty sentences in your corpus → words loop (like if len(tokens) > 0),you are not preserving one-to-one alignment between corpus and words. 
# So if words[4291] and words[5170] are empty, that tells us: these indices in words do not correspond to the same indices in corpus anymore.
# If you want to check which corpus entries created empty words,you must loop together like this:
empty_indices = []
for i, sent in enumerate(corpus):
    sent_token = sent_tokenize(sent)
    for s in sent_token:
        tokens = simple_preprocess(s)
        if len(tokens) == 0:
            empty_indices.append(i)


In [ ]:
empty_indices

In [ ]:
for i in [4293, 5173]:
    print(f"Index {i}: {repr(corpus[i])}")

##### If you see in the above they are very short messages 'g w r' and 'u'. Each contains a one or a few single-letter “words”. 
What simple_preprocess() does:

gensim.utils.simple_preprocess() is not just a basic split —
1. it removes very short tokens by default.
2. By design, it ignores tokens that:  

Are shorter than 2 characters (default min_len=2)  

Or longer than 15 characters (default max_len=15)  


In [ ]:
# we get 5569 because 3 messages became empty after cleaning.
len(words)

In [87]:
# Let's train our own Word2Vec model.
import gensim
# By default,min_count is 5, so words which are having frequency less than 5 will be ignored.
# My model is trained on the words
model=gensim.models.Word2Vec(words) # we can mention parameters like vector size, window size, min count etc. By default vector size is 100, window size is 5 and min count is 5.

#### 🧩 1️⃣ “100 dimensions” — are those also the words in my vocabulary?
1. ❌ No — the 100 dimensions are not words. They are number of numeric features that capture relationships (like gender, tense, topic, etc.)
2. They are numerical features (latent semantic dimensions) that represent meaning or context of words — not actual words themselves.
3. These 100 numbers don’t correspond to specific words.Instead, they describe abstract properties — like:

a. masculine/feminine axis   
b. royalty/commoner axis  
c. age, emotion, topic, etc.  

The model learns these patterns automatically while training.



In [82]:
# It will give the count of the words in the vocabulary and their dimension(means no.of columns)
# since we have not mentioned any parameters, by default vector size is 100. so number of columns is 100.
model.wv.vectors.shape 

(1721, 100)

In [ ]:
# To get all the vocabulary words in the model
# Since I mentioned min_count as 5, words which are having frequency less than 5 are ignored.
model.wv.index_to_key

#### 🧩 2️⃣ Should model.wv.vectors.shape and model.corpus_count be the same?
❌ No, they are not the same thing — and they almost never match.
| Attribute                   | Meaning                                                    |
| --------------------------- | ---------------------------------------------------------- |
| `model.wv.vectors.shape[0]` | Number of **unique words in the vocabulary** (rows)        |
| `model.corpus_count`        | Number of **sentences** (or “documents”) used for training |

🧠 Analogy

Think of Word2Vec as a language school:  

corpus_count → how many sentences it studied.  

wv.vectors.shape[0] → how many unique words it learned from them.  

wv.vectors.shape[1] → how many traits each word has learned (like tone, tense, meaning).  


In [ ]:
# It indicates the number of sentences (or “documents”) used for training
model.corpus_count

In [ ]:
# model.epochs means it tells how many times your Word2Vec model iterated over the entire training corpus during training
model.epochs

In [ ]:
# Check the similar words
model.wv.most_similar('good')

In [84]:
model.wv['good'].shape

(100,)

In [85]:
words[0]

['go',
 'until',
 'jurong',
 'point',
 'crazy',
 'available',
 'only',
 'in',
 'bugis',
 'great',
 'world',
 'la',
 'buffet',
 'cine',
 'there',
 'got',
 'amore',
 'wat']

##### What is model.wv in Word2Vec?
| Part       | Description                                                             |
| ---------- | ----------------------------------------------------------------------- |
| `model.wv` | **Word vectors** — this is your vocabulary + learned embeddings         |
| `model`    | Full model (includes training settings, negative sampling tables, etc.) |

✅ So yes — model.wv is the learned vocabulary of your Word2Vec model.
1. It stores all unique words that appeared in your training corpus (subject to min_count).
2. It also stores the vector representation for each of those words.

##### How we can get the NaNs?
If a sentence has no valid words in model.wv, like "12345", "!!!", "###", etc.
Then the list vectors becomes empty: vectors=[]  
and   

np.mean(vector,axis=0) returns nan.



In [100]:
# Since we have 100 dimesions for every word of word[0] similarly for all words in the words list.We need to take average of all word vectors to represent the entire sentence.
# model.wv- 
import numpy as np
def avg_word2vec(tokens,model):
    # Iterates through each word in the sentence and retrieves its corresponding word vector from the Word2Vec model (model.wv).Collects these vectors into a list called vectors.
    vectors=[model.wv[word] for word in tokens if word in model.wv]
    if len(vectors)>0:
       return np.mean(vectors, axis=0)
    else:
       # Suppose if the words in the sentence are not present in the vocabulary of the model i.e model.wv, then the list vectors becomes empty and if we do np.mean(vectors, axis=0) it returns a Nan value. To avoid that we can return a zero vector of same dimension as the model's word vectors.
       return np.zeros(model.vector_size) # Handle the case where no words are found in the model's vocabulary

In [ ]:
pip install tqdm

##### tqdm:
tqdm is a python library for progress bars. It shows you a real-time progress indicator in the console or notebook while loops are running.


In [ ]:
from tqdm import tqdm

In [ ]:
# Apply the avg_word2vec function to each sentence in the words list to get the average word vectors for all sentences.
import numpy as np
# X is a list of vectors
X=[]
for i in tqdm(range(len(words))):
    X.append(avg_word2vec(words[i],model))
# Converts X into a Numpy array
# X is a list of vectors. Not compatible with ML models. so convert it into a 2D NumPy array.
X=np.array(X)

100%|██████████| 5569/5569 [00:00<00:00, 17723.21it/s]


In [105]:
len(X)

5569

In [106]:
type(X)

numpy.ndarray

In [107]:
len(X)  # number of sentences

5569

In [108]:
messages.shape

(5572, 2)

In [ ]:
y= messages['label'].map({'ham': 0, 'spam': 1}).values

In [ ]:
len(y)

#### Execution of the below code:
| Step | Expression                          | What It Does                                                    | Result Type  |
| ---- | ----------------------------------- | --------------------------------------------------------------- | ------------ |
| 1    | `map(lambda x: len(x) > 0, corpus)` | Checks which corpus entries are non-empty                       | list of bool |
| 2    | `messages[...]`                     | Filters DataFrame to keep only rows with non-empty cleaned text | DataFrame    |
| 3    | `y['label']`                        | Selects the label column                                        | Series       |
| 4    | `.map({'ham': 0, 'spam': 1})`       | Converts text labels to numbers                                 | Series (int) |
| 5    | `.values`                           | Converts Series → NumPy array                                   | ndarray      |


In [ ]:
# y is considering all the sentences but after cleaning like applying the regular expression '[^a-zA-Z]' we got the total 5569 sentences lets add that filter to y.
# messages is the dataframe with 2 columns message and label and y stores the updated dataframe  where the corresponding corpus entry was non-empty.
y=messages[list(map(lambda x:len(x)>0, corpus))] # This selects only the rows where the corresponding corpus entry was non-empty.
# From the filtered DataFrame y, we take the label column and replaces the string labels using a dictionary: 'ham' → 0 and 'spam' → 1
# .values - this converts the pandas series to a numpy array.
y= y['label'].map({'ham': 0, 'spam': 1}).values          


In [80]:
len(y)

5569

In [81]:
y.shape

(5569,)

In [111]:
X.shape

(5569, 100)

In [109]:
X[0].shape

(100,)

In [112]:
# Final independent features
# Convert the entire 2D array X into a DataFrame
df = pd.DataFrame(X)
print(df.isnull().sum().sum())  # should be 0


0


In [113]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.184675,0.149868,0.089056,0.094514,0.090070,-0.514925,0.145919,0.472682,-0.259969,-0.115657,...,0.329387,0.161341,0.075355,0.020196,0.444074,0.183508,0.185545,-0.169584,0.147358,0.017244
1,-0.170940,0.132376,0.075826,0.081203,0.084306,-0.449808,0.116743,0.418098,-0.227575,-0.095866,...,0.293413,0.133892,0.060903,0.010904,0.376821,0.155787,0.162528,-0.157472,0.136299,0.008651
2,-0.196712,0.162979,0.097607,0.114731,0.072596,-0.549484,0.144973,0.468559,-0.273950,-0.140274,...,0.324815,0.165711,0.076328,0.008008,0.459194,0.167880,0.138783,-0.200509,0.176054,0.031851
3,-0.253898,0.202698,0.116573,0.127540,0.123346,-0.701212,0.194435,0.650995,-0.357901,-0.150240,...,0.450066,0.215079,0.098566,0.032351,0.593383,0.255118,0.269659,-0.231937,0.202422,0.013255
4,-0.217036,0.161471,0.106043,0.102467,0.113860,-0.591734,0.160089,0.549833,-0.304921,-0.132945,...,0.382365,0.179177,0.084932,0.030410,0.499131,0.216194,0.220312,-0.203990,0.161932,0.009948


In [114]:
df['Output']=y

In [115]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,Output
0,-0.184675,0.149868,0.089056,0.094514,0.090070,-0.514925,0.145919,0.472682,-0.259969,-0.115657,...,0.161341,0.075355,0.020196,0.444074,0.183508,0.185545,-0.169584,0.147358,0.017244,0
1,-0.170940,0.132376,0.075826,0.081203,0.084306,-0.449808,0.116743,0.418098,-0.227575,-0.095866,...,0.133892,0.060903,0.010904,0.376821,0.155787,0.162528,-0.157472,0.136299,0.008651,0
2,-0.196712,0.162979,0.097607,0.114731,0.072596,-0.549484,0.144973,0.468559,-0.273950,-0.140274,...,0.165711,0.076328,0.008008,0.459194,0.167880,0.138783,-0.200509,0.176054,0.031851,1
3,-0.253898,0.202698,0.116573,0.127540,0.123346,-0.701212,0.194435,0.650995,-0.357901,-0.150240,...,0.215079,0.098566,0.032351,0.593383,0.255118,0.269659,-0.231937,0.202422,0.013255,0
4,-0.217036,0.161471,0.106043,0.102467,0.113860,-0.591734,0.160089,0.549833,-0.304921,-0.132945,...,0.179177,0.084932,0.030410,0.499131,0.216194,0.220312,-0.203990,0.161932,0.009948,0


In [119]:
# INdependent feature
# axis=1 → tells pandas to drop a column (instead of a row)
X=df.drop('Output', axis=1)

In [120]:
X

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.184675,0.149868,0.089056,0.094514,0.090070,-0.514925,0.145919,0.472682,-0.259969,-0.115657,...,0.329387,0.161341,0.075355,0.020196,0.444074,0.183508,0.185545,-0.169584,0.147358,0.017244
1,-0.170940,0.132376,0.075826,0.081203,0.084306,-0.449808,0.116743,0.418098,-0.227575,-0.095866,...,0.293413,0.133892,0.060903,0.010904,0.376821,0.155787,0.162528,-0.157472,0.136299,0.008651
2,-0.196712,0.162979,0.097607,0.114731,0.072596,-0.549484,0.144973,0.468559,-0.273950,-0.140274,...,0.324815,0.165711,0.076328,0.008008,0.459194,0.167880,0.138783,-0.200509,0.176054,0.031851
3,-0.253898,0.202698,0.116573,0.127540,0.123346,-0.701212,0.194435,0.650995,-0.357901,-0.150240,...,0.450066,0.215079,0.098566,0.032351,0.593383,0.255118,0.269659,-0.231937,0.202422,0.013255
4,-0.217036,0.161471,0.106043,0.102467,0.113860,-0.591734,0.160089,0.549833,-0.304921,-0.132945,...,0.382365,0.179177,0.084932,0.030410,0.499131,0.216194,0.220312,-0.203990,0.161932,0.009948
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5564,-0.223848,0.192581,0.124638,0.131275,0.095183,-0.619041,0.178100,0.550970,-0.315842,-0.151573,...,0.387403,0.196807,0.099445,0.013497,0.541170,0.199318,0.181446,-0.214891,0.190730,0.037260
5565,-0.225151,0.173134,0.104921,0.117431,0.109292,-0.618269,0.171146,0.563637,-0.327013,-0.136646,...,0.399320,0.196687,0.074456,0.019759,0.520700,0.212317,0.205315,-0.222403,0.182648,0.023028
5566,-0.251484,0.201482,0.112697,0.115808,0.112633,-0.681657,0.193501,0.634659,-0.345379,-0.158240,...,0.440722,0.224199,0.105644,0.038959,0.591623,0.254272,0.264226,-0.215579,0.187010,0.016739
5567,-0.227057,0.179454,0.107558,0.113808,0.103727,-0.625560,0.171240,0.572667,-0.315513,-0.147044,...,0.393685,0.196317,0.097647,0.030170,0.536607,0.224867,0.220657,-0.205249,0.172123,0.013329


In [121]:
df

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,Output
0,-0.184675,0.149868,0.089056,0.094514,0.090070,-0.514925,0.145919,0.472682,-0.259969,-0.115657,...,0.161341,0.075355,0.020196,0.444074,0.183508,0.185545,-0.169584,0.147358,0.017244,0
1,-0.170940,0.132376,0.075826,0.081203,0.084306,-0.449808,0.116743,0.418098,-0.227575,-0.095866,...,0.133892,0.060903,0.010904,0.376821,0.155787,0.162528,-0.157472,0.136299,0.008651,0
2,-0.196712,0.162979,0.097607,0.114731,0.072596,-0.549484,0.144973,0.468559,-0.273950,-0.140274,...,0.165711,0.076328,0.008008,0.459194,0.167880,0.138783,-0.200509,0.176054,0.031851,1
3,-0.253898,0.202698,0.116573,0.127540,0.123346,-0.701212,0.194435,0.650995,-0.357901,-0.150240,...,0.215079,0.098566,0.032351,0.593383,0.255118,0.269659,-0.231937,0.202422,0.013255,0
4,-0.217036,0.161471,0.106043,0.102467,0.113860,-0.591734,0.160089,0.549833,-0.304921,-0.132945,...,0.179177,0.084932,0.030410,0.499131,0.216194,0.220312,-0.203990,0.161932,0.009948,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5564,-0.223848,0.192581,0.124638,0.131275,0.095183,-0.619041,0.178100,0.550970,-0.315842,-0.151573,...,0.196807,0.099445,0.013497,0.541170,0.199318,0.181446,-0.214891,0.190730,0.037260,1
5565,-0.225151,0.173134,0.104921,0.117431,0.109292,-0.618269,0.171146,0.563637,-0.327013,-0.136646,...,0.196687,0.074456,0.019759,0.520700,0.212317,0.205315,-0.222403,0.182648,0.023028,0
5566,-0.251484,0.201482,0.112697,0.115808,0.112633,-0.681657,0.193501,0.634659,-0.345379,-0.158240,...,0.224199,0.105644,0.038959,0.591623,0.254272,0.264226,-0.215579,0.187010,0.016739,0
5567,-0.227057,0.179454,0.107558,0.113808,0.103727,-0.625560,0.171240,0.572667,-0.315513,-0.147044,...,0.196317,0.097647,0.030170,0.536607,0.224867,0.220657,-0.205249,0.172123,0.013329,0


In [122]:
y=df['Output']

In [123]:
# Train test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20)

In [124]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
4869,-0.221435,0.147823,0.100683,0.074971,0.131486,-0.566019,0.141018,0.539932,-0.293998,-0.127808,...,0.389135,0.168919,0.072802,0.042755,0.458136,0.238732,0.229641,-0.190284,0.120561,0.010098
2509,-0.190371,0.158153,0.088873,0.092478,0.097428,-0.538097,0.149127,0.502132,-0.274447,-0.119062,...,0.346429,0.168213,0.077290,0.020860,0.461935,0.198700,0.200154,-0.174497,0.148450,0.008869
53,-0.252700,0.190297,0.117706,0.112766,0.127885,-0.672676,0.180623,0.625596,-0.341050,-0.154518,...,0.443120,0.207461,0.096214,0.028234,0.568651,0.250724,0.251985,-0.228703,0.176215,0.019859
2804,-0.230341,0.177645,0.109295,0.127031,0.112223,-0.627240,0.158965,0.557036,-0.317636,-0.149579,...,0.386982,0.194045,0.086125,0.014686,0.516601,0.205334,0.170706,-0.231239,0.185476,0.032180
2891,-0.222548,0.178217,0.100847,0.105852,0.105742,-0.621960,0.170569,0.577343,-0.311346,-0.141219,...,0.403997,0.195869,0.088569,0.018641,0.527501,0.223433,0.223891,-0.204896,0.166158,0.010368


In [125]:
y_train

4869    0
2509    0
53      0
2804    0
2891    0
       ..
3511    0
615     0
681     0
5328    0
4205    0
Name: Output, Length: 4455, dtype: int64

In [126]:
from sklearn.ensemble import RandomForestClassifier
classifier= RandomForestClassifier()

In [128]:
classifier.fit(X_train,y_train)

RandomForestClassifier()

In [129]:
y_pred=classifier.predict(X_test)

In [131]:
# Model Evaluation
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report
confusion_mtx = confusion_matrix(y_test, y_pred)
print(f"confusion matrix\n: {confusion_mtx}")
accuracy = accuracy_score(y_test, y_pred)
print(f" Accuracy : {accuracy}")
precision = precision_score(y_test, y_pred)
print(f" Precision : {precision}")
recall = recall_score(y_test, y_pred)
print(f" Recall : {recall}")
f1 = f1_score(y_test, y_pred)
print(f" F1 Score : {f1}")

confusion matrix
: [[957  17]
 [ 19 121]]
 Accuracy : 0.9676840215439856
 Precision : 0.8768115942028986
 Recall : 0.8642857142857143
 F1 Score : 0.8705035971223022


In [132]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       974
           1       0.88      0.86      0.87       140

    accuracy                           0.97      1114
   macro avg       0.93      0.92      0.93      1114
weighted avg       0.97      0.97      0.97      1114

